In [1]:
%reload_ext autoreload
%autoreload 2
import glob
import importlib
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import scipy.io.wavfile as wv
import scipy.signal as sig
import wave

from datetime import datetime
from keras.models import Sequential, load_model
from keras.layers import Dense

from util import util
from util import WavFileParts
from util.logUtil import LOG, LOG_HEADER
from util.confusionMatrix import ConfusionMatrix


/opt/miniconda2/envs/py36h2o/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


### globale settings

In [2]:
classes = ['music', 'voice', 'environment']

macDir = '/Volumes/SAA_DATA/datasets/'
winDir = 'E:/SAA_DATA/'
osDir = macDir
recordingDir = osDir + '/localizationRecordings'

if osDir == winDir:
    storageFolder = 'E:/SAA_DATA/storedData/'
else:
    storageFolder = '/Users/etto/Desktop/storedData/'

baseSrcDir = osDir + 'localizationFiles/20171025AllExtractionsMic4'
orgWavDirs1 = ['G428_0.0_1.4',
              'G527_0.5_1.4',
              'Studio_2.0_4.2'
              ]

orgWavDirs2 = ['G428_2.1_2.4',
              'G527_1.2_5.8',
              'Studio_3.0_2.0'
              ]

orgsG428 = ['G428_0.0_1.4','G428_2.1_2.4']
orgsG527 = ['G527_0.5_1.4','G527_1.2_5.8']
orgsStudio = ['Studio_2.0_4.2','Studio_3.0_2.0']

NFFT = 1024

chunksBaseDir = 'chunks'
rooms = ['Studio', 'G428', 'G527']

### utility functies

In [3]:
def readSoundChunksDynamic(moduleString):
    chunks = importlib.import_module(moduleString).soundChunks
    wfPts = []
    for jsonString in chunks:
        wfPts.append(WavFileParts.WavFilePartFromJson(jsonString))
    return wfPts

def timeFunction(func):
    """
    Aanroep: bijv. fpc = timeFunction(lambda: getFilesPerCategory(srcDir))
    """
    startTime = datetime.now()
    print('Start: ' + startTime.strftime('%H:%M:%S') + '\n=================')

    res = func()
    
    endTime = datetime.now()
    print('\n=================\nEnd: ' + endTime.strftime('%H:%M:%S'))
    print('Time taken: '),
    print(endTime - startTime)
    print()
    
    return res
    
def storeTestData(allSpectros, allClasses, storageName, keyName):
    filename = storageFolder + storageName + '.hd5'
    df = pd.DataFrame(allSpectros)
    df.to_hdf(path_or_buf=filename, key='spectros_' + keyName)

    df = pd.DataFrame(allClasses)
    df.to_hdf(path_or_buf=filename, key='classes_' + keyName)

def retrieveTestData(storageName, keyName):
    filename = storageFolder + storageName + '.hd5'
    specDf = pd.read_hdf(path_or_buf=filename, key='spectros_' + keyName)
    classesDf = pd.read_hdf(path_or_buf=filename, key='classes_' + keyName)
    return specDf.values, classesDf.values
    

### functies tbv trainen

In [4]:
# Maakt een dictionary aan; per categorie alle files (volledig pad) uit de srcDir
# srcDir is een van de orgWavDirs, bijvoorbeeld
#    localizationFiles/20171025AllExtractionsMic4/G428_0.0_1.4
def getFilesPerCategory(srcDir):
    filesPerCategory = {}
    for catDirLong in glob.glob(srcDir + '/*'):
        catDir = catDirLong.replace('\\', '/')
        catDir = catDir.replace(srcDir + '/', '')

        filesPerCategory[catDir] = []
        for filename in glob.glob(catDirLong + '/*'):
            filename = filename.replace('\\','/')
            filesPerCategory[catDir].append(filename)
    return filesPerCategory

def getFilesPerCatFromMultipleDirs(srcDirs, srcDirsBase=''):
    filesPerCat = {}
    for dirName in srcDirs:
        srcDir = srcDirsBase + '/' + dirName
        fpcNw = getFilesPerCategory(srcDir)
        if not filesPerCat:
            filesPerCat = fpcNw
        else:
            for key in filesPerCat:
                filesPerCat[key] += fpcNw[key]
    return filesPerCat
        

In [5]:
# Maakt een dictionary aan; per categorie de spectrogrammen
def getSpectrosFromFilesPerCategory(filesPerCategory):
    spectros = {}
    for clz in classes:
        spectros[clz] = []
        for filename in filesPerCategory[clz]:
            fs, signal = wv.read(filename)
            freq_array, segment_times, spectrogram = sig.spectrogram(x=signal, fs=fs, nfft=NFFT, noverlap=0)
            spectros[clz].append(spectrogram.T)
    return spectros

In [6]:
def getClassLengths(spectrosPerCat):
    clzLengths = {}
    for clz in classes:
        clzLengths[clz] = sum([np.shape(lst)[0] for lst in spectrosPerCat[clz]])
    return clzLengths

In [7]:
# verwacht invoer van getSpectrosFromFilesPerCategory
# levert traindata op (X_train en Y_train)
def createTrainDataFromSpectros(spectrosPerCat, clzLengths):
    X_train = np.concatenate(spectrosPerCat[classes[0]], axis=0)
    for i in range(1, len(classes)):
        nwSpectros = np.concatenate(spectrosPerCat[classes[i]], axis=0)
        X_train = np.concatenate((X_train,nwSpectros), axis=0)
    
    # one-hot encoding voor Y_train
    nrFiles = clzLengths[classes[0]]
    Y_train = np.array((np.ones(nrFiles),np.zeros(nrFiles), np.zeros(nrFiles))).T

    nrFiles = clzLengths[classes[1]]
    Y_train_nw = np.array((np.zeros(nrFiles), np.ones(nrFiles), np.zeros(nrFiles))).T
    Y_train = np.concatenate((Y_train, Y_train_nw),axis=0)

    nrFiles = clzLengths[classes[2]]
    Y_train_nw = np.array((np.zeros(nrFiles), np.zeros(nrFiles), np.ones(nrFiles))).T
    Y_train = np.concatenate((Y_train, Y_train_nw),axis=0)
    
    return X_train, Y_train

### deep learning model

In [8]:
def create_model(layersizes):
    # create model
    model = Sequential()
    model.add(Dense(layersizes[0], input_dim=513, activation='relu'))
    for lsize in layersizes[1:]:
        model.add(Dense(lsize, activation='relu'))
    model.add(Dense(3, activation='softmax'))
    # Compile model
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [9]:
def getModelFileName(modelFilePath, baseModelFilename, layers, nrEpochs):
    modelFilename = modelFilePath + baseModelFilename
    for lsize in layers:
        modelFilename = '{}_{}'.format(modelFilename, lsize)
    modelFilename += 'ep{}'.format(nrEpochs)
    modelFilename += '.hd5'
    return modelFilename

In [10]:
def getModelName(baseModelFilename, layers, nrEpochs):
    modelFilename = baseModelFilename
    for lsize in layers:
        modelFilename = '{}_{}'.format(modelFilename, lsize)
    modelFilename += 'ep{}'.format(nrEpochs)
    return modelFilename

In [11]:
def train_model(X_train, Y_train, layers, nrEpochs, modelFilePath, baseModelFilename):
    soundModel = create_model(layers)
    history = timeFunction(lambda: soundModel.fit(X_train,Y_train, epochs=nrEpochs, shuffle=True, verbose=1))
    soundModel.save(getModelFileName(modelFilePath, baseModelFilename, layers, nrEpochs))
    plt.plot(history.history['loss'])
    plt.show()

In [12]:
def evaluate_model(X_test, realClasses, layers, nrEpochs, modelFilePath, baseModelFilename):
    soundModel = load_model(getModelFileName(modelFilePath, baseModelFilename, layers, nrEpochs))

    # predicted classes
    predictions = soundModel.predict(X_test)
    predClasses = predictions.argmax(axis=1)

    matrix = ConfusionMatrix(classes)
    for vals in zip(realClasses, predClasses):
        matrix.add(int(vals[0]), int(vals[1]), 1)
    LOG(matrix.toString(),True)
    LOG('', True)
    LOG(matrix.toF1String(), True)
    return matrix

In [13]:
def train_and_evaluate_per_epoch(X_train, Y_train, realClasses, layers, nrEpochs, modelFilePath, baseModelFilename):
    soundModel = create_model(layers)
    for epNr in range(1, nrEpochs+1):
        LOG('\n*****************\n* Epoch nr {}\n*****************\n'.format(epNr), True)
        soundModel.fit(X_train,Y_train, epochs=1, shuffle=True, verbose=1)
        soundModel.save(getModelFileName(modelFilePath, baseModelFilename, layers, epNr))
        evaluate_model(X_train, realClasses, layers, epNr, modelFilePath, baseModelFilename)

### functies tbv testen

In [14]:
def createAndStoreTestData(wavFileParts, baseDir, fileDate, micNr, storeFilename, keyName):
    allSpectros = np.array([])
    allClasses = np.array([])

    for wfPt in wavFileParts: #type: WavFilePart
        if not 'Gunshot' in wfPt.getSoundType():
            filename = baseDir + '/{:d}_{:d}_mono{:d}.wav'.format(fileDate, wfPt.fileNr, micNr)
            fs, signal = wv.read(filename)

            classNr = classes.index(wfPt.getSoundType().lower())
            for soundChunk in wfPt.getSoundChunks(micNr):
                startFrame = int(soundChunk[0] * fs)
                endFrame = int(soundChunk[1] * fs)

                sigChunk = signal[startFrame: endFrame]
                freq_array, segment_times, spectrogram = sig.spectrogram(x=sigChunk, fs=fs, nfft=NFFT, noverlap=0)
                if len(allSpectros) == 0:
                    allSpectros = spectrogram.T
                else:
                    allSpectros = np.append(allSpectros, spectrogram.T, axis=0)
                allClasses = np.append(allClasses, classNr * np.ones(len(segment_times)))
                
    storeTestData(allSpectros, allClasses, storeFilename, keyName)


In [15]:
def getTrainDataFromFolders(orgWavDirs):
    fpc = getFilesPerCatFromMultipleDirs(orgWavDirs, baseSrcDir)
    spcs = getSpectrosFromFilesPerCategory(fpc)
    clzLengths = getClassLengths(spcs)
    X_train, Y_train = createTrainDataFromSpectros(spcs, clzLengths)
    # real train classes
    realTrainClasses = np.concatenate((np.zeros(clzLengths[classes[0]]), 
                                  np.ones(clzLengths[classes[1]]), 
                                  2*np.ones(clzLengths[classes[2]])))
    return X_train, Y_train, realTrainClasses


### trainen van model obv: G428

In [16]:
baseModelFilename = '20180817_orgG428'
modelFilePath = storageFolder
logPrefix = 'Orgs G428'
orgWavDirs = orgsG428
layersList = [[100, 20], [400, 250, 100, 20], [400, 300, 200, 100, 50, 20, 10], 
              [450, 400, 350, 300, 250, 200, 150, 100, 50, 21]]
nrEpochs = 5
nrsEpochs = [1,2,3,4,5]

In [17]:
# Train voor alle lagen
X_train, Y_train, realTrainClasses = getTrainDataFromFolders(orgWavDirs)
for layers in layersList:
    LOG_HEADER(logPrefix + ', lagen: {}'.format(str(layers)), True)
    train_and_evaluate_per_epoch(X_train, Y_train, realTrainClasses, layers, nrEpochs, modelFilePath, baseModelFilename)

####################################
#
# Orgs G428, lagen: [100, 20]
#
####################################


*****************
* Epoch nr 1
*****************

Epoch 1/1
375667/375667 [==============================] - 28s 74us/step - loss: 0.1644 - acc: 0.9401
                   m       v       e |  sens   acc
music         115033    6200     108 |  0.95  0.98
voice           2330  116419    2884 |  0.96  0.96
environment      261    2577  129855 |  0.98  0.98
--------------------------------------
prec            0.98    0.93    0.98

F1 overall: 0.96
F1 music: 0.96
F1 voice: 0.94
F1 environment: 0.98


*****************
* Epoch nr 2
*****************

Epoch 1/1
375667/375667 [==============================] - 28s 74us/step - loss: 0.0966 - acc: 0.9642
                   m       v       e |  sens   acc
music         115915    5370      56 |  0.96  0.98
voice           1976  116746    2911 |  0.96  0.97
environment      257    1266  131170 |  0.99  0.99
-------------------------------

375667/375667 [==============================] - 199s 530us/step - loss: 0.1496 - acc: 0.9464
                   m       v       e |  sens   acc
music         115554    5622     165 |  0.95  0.97
voice           3732  114589    3312 |  0.94  0.96
environment      103    1496  131094 |  0.99  0.99
--------------------------------------
prec            0.97    0.94    0.97

F1 overall: 0.96
F1 music: 0.96
F1 voice: 0.94
F1 environment: 0.98


*****************
* Epoch nr 2
*****************

Epoch 1/1
375667/375667 [==============================] - 162s 430us/step - loss: 0.0993 - acc: 0.9660
                   m       v       e |  sens   acc
music         113634    7532     175 |  0.94  0.98
voice            811  118613    2209 |  0.98  0.97
environment       84    2346  130263 |  0.98  0.99
--------------------------------------
prec            0.99    0.92    0.98

F1 overall: 0.96
F1 music: 0.96
F1 voice: 0.95
F1 environment: 0.98


*****************
* Epoch nr 3
*****************



In [18]:
# Test alle modellen

# model parameters
modelFilePath = storageFolder
layerss = layersList

# test data
micNr = 4
testFileNames = ['testData_G428', 'testData_G527', 'testData_Studio']
keyname = 'mic{}'.format(micNr)

for testFileName in testFileNames:
    for layers in layerss:
        for nrEpochs in nrsEpochs:
            testSpecs, testClasses = retrieveTestData(testFileName, keyname)

            LOG_HEADER(getModelName(baseModelFilename, layers, nrEpochs) 
                       + '\n# ' + keyname
                       + '\n# ' + testFileName,
                       True)
            matrix = evaluate_model(testSpecs, testClasses, layers, nrEpochs, modelFilePath, baseModelFilename)


####################################
#
# 20180817_orgG428_100_20ep1
# mic4
# testData_G428
#
####################################

                   m       v       e |  sens   acc
music          19077    1516      46 |  0.92  0.96
voice            339   13800     297 |  0.96  0.95
environment       22     147   13056 |  0.99  0.99
--------------------------------------
prec            0.98    0.89    0.97

F1 overall: 0.95
F1 music: 0.95
F1 voice: 0.92
F1 environment: 0.98

####################################
#
# 20180817_orgG428_100_20ep2
# mic4
# testData_G428
#
####################################

                   m       v       e |  sens   acc
music          19388    1212      39 |  0.94  0.97
voice            285   13923     228 |  0.96  0.96
environment       34     103   13088 |  0.99  0.99
--------------------------------------
prec            0.98    0.91    0.98

F1 overall: 0.96
F1 music: 0.96
F1 voice: 0.94
F1 environment: 0.98

####################################
#

                   m       v       e |  sens   acc
music          19743     862      34 |  0.96  0.97
voice            450   13809     177 |  0.96  0.97
environment       65     192   12968 |  0.98  0.99
--------------------------------------
prec            0.97    0.93    0.98

F1 overall: 0.96
F1 music: 0.97
F1 voice: 0.94
F1 environment: 0.98

####################################
#
# 20180817_orgG428_450_400_350_300_250_200_150_100_50_21ep4
# mic4
# testData_G428
#
####################################

                   m       v       e |  sens   acc
music          19506    1105      28 |  0.95  0.97
voice            461   13771     204 |  0.95  0.96
environment       26     154   13045 |  0.99  0.99
--------------------------------------
prec            0.98    0.92    0.98

F1 overall: 0.96
F1 music: 0.96
F1 voice: 0.93
F1 environment: 0.98

####################################
#
# 20180817_orgG428_450_400_350_300_250_200_150_100_50_21ep5
# mic4
# testData_G428
#
##############

                   m       v       e |  sens   acc
music           8638     264      15 |  0.97  0.97
voice            110    2753       3 |  0.96  0.97
environment        0      12    2479 |  1.00  1.00
--------------------------------------
prec            0.99    0.91    0.99

F1 overall: 0.97
F1 music: 0.98
F1 voice: 0.93
F1 environment: 0.99

####################################
#
# 20180817_orgG428_450_400_350_300_250_200_150_100_50_21ep1
# mic4
# testData_G527
#
####################################

                   m       v       e |  sens   acc
music           8276     619      22 |  0.93  0.94
voice            229    2633       4 |  0.92  0.94
environment        1       6    2484 |  1.00  1.00
--------------------------------------
prec            0.97    0.81    0.99

F1 overall: 0.93
F1 music: 0.95
F1 voice: 0.86
F1 environment: 0.99

####################################
#
# 20180817_orgG428_450_400_350_300_250_200_150_100_50_21ep2
# mic4
# testData_G527
#
##############

####################################
#
# 20180817_orgG428_400_300_200_100_50_20_10ep2
# mic4
# testData_Studio
#
####################################

                   m       v       e |  sens   acc
music          33936   26021    3638 |  0.53  0.79
voice             16   36725    1163 |  0.97  0.78
environment        7    4404   37926 |  0.90  0.94
--------------------------------------
prec            1.00    0.55    0.89

F1 overall: 0.76
F1 music: 0.70
F1 voice: 0.70
F1 environment: 0.89

####################################
#
# 20180817_orgG428_400_300_200_100_50_20_10ep3
# mic4
# testData_Studio
#
####################################

                   m       v       e |  sens   acc
music          39682   20370    3543 |  0.62  0.83
voice            254   36028    1622 |  0.95  0.83
environment       10    1726   40601 |  0.96  0.95
--------------------------------------
prec            0.99    0.62    0.89

F1 overall: 0.81
F1 music: 0.77
F1 voice: 0.75
F1 environment: 0.92

### trainen van model obv: G527

In [19]:
baseModelFilename = '20180817_orgG527'
modelFilePath = storageFolder
logPrefix = 'Orgs G527'
orgWavDirs = orgsG527
layersList = [[100, 20], [400, 250, 100, 20], [400, 300, 200, 100, 50, 20, 10], 
              [450, 400, 350, 300, 250, 200, 150, 100, 50, 21]]
nrEpochs = 5
nrsEpochs = [1,2,3,4,5]

In [20]:
# Train voor alle lagen
X_train, Y_train, realTrainClasses = getTrainDataFromFolders(orgWavDirs)
for layers in layersList:
    LOG_HEADER(logPrefix + ', lagen: {}'.format(str(layers)), True)
    train_and_evaluate_per_epoch(X_train, Y_train, realTrainClasses, layers, nrEpochs, modelFilePath, baseModelFilename)

####################################
#
# Orgs G527, lagen: [100, 20]
#
####################################


*****************
* Epoch nr 1
*****************

Epoch 1/1
378662/378662 [==============================] - 66s 175us/step - loss: 0.1802 - acc: 0.9348
                   m       v       e |  sens   acc
music         115048    7046     211 |  0.94  0.97
voice           2997  116837    2780 |  0.95  0.96
environment      286    3564  129893 |  0.97  0.98
--------------------------------------
prec            0.97    0.92    0.98

F1 overall: 0.96
F1 music: 0.96
F1 voice: 0.93
F1 environment: 0.97


*****************
* Epoch nr 2
*****************

Epoch 1/1
378662/378662 [==============================] - 61s 161us/step - loss: 0.1092 - acc: 0.9600
                   m       v       e |  sens   acc
music         117092    5124      89 |  0.96  0.98
voice           2866  117857    1891 |  0.96  0.96
environment      292    3450  130001 |  0.97  0.98
-----------------------------

378662/378662 [==============================] - 201s 532us/step - loss: 0.1581 - acc: 0.9442
                   m       v       e |  sens   acc
music         115931    6107     267 |  0.95  0.98
voice           2673  117627    2314 |  0.96  0.96
environment      169    2647  130927 |  0.98  0.99
--------------------------------------
prec            0.98    0.93    0.98

F1 overall: 0.96
F1 music: 0.96
F1 voice: 0.94
F1 environment: 0.98


*****************
* Epoch nr 2
*****************

Epoch 1/1
378662/378662 [==============================] - 197s 520us/step - loss: 0.1086 - acc: 0.9622
                   m       v       e |  sens   acc
music         116915    5266     124 |  0.96  0.98
voice           2342  118102    2170 |  0.96  0.97
environment      295    2313  131135 |  0.98  0.99
--------------------------------------
prec            0.98    0.94    0.98

F1 overall: 0.97
F1 music: 0.97
F1 voice: 0.95
F1 environment: 0.98


*****************
* Epoch nr 3
*****************



In [21]:
# Test alle modellen

# model parameters
modelFilePath = storageFolder
layerss = layersList

# test data
micNr = 4
testFileNames = ['testData_G428', 'testData_G527', 'testData_Studio']
keyname = 'mic{}'.format(micNr)

for testFileName in testFileNames:
    for layers in layerss:
        for nrEpochs in nrsEpochs:
            testSpecs, testClasses = retrieveTestData(testFileName, keyname)

            LOG_HEADER(getModelName(baseModelFilename, layers, nrEpochs) 
                       + '\n# ' + keyname
                       + '\n# ' + testFileName,
                       True)
            matrix = evaluate_model(testSpecs, testClasses, layers, nrEpochs, modelFilePath, baseModelFilename)


####################################
#
# 20180817_orgG527_100_20ep1
# mic4
# testData_G428
#
####################################

                   m       v       e |  sens   acc
music          19645     957      37 |  0.95  0.96
voice            708   13365     363 |  0.93  0.96
environment      235     109   12881 |  0.97  0.98
--------------------------------------
prec            0.95    0.93    0.97

F1 overall: 0.95
F1 music: 0.95
F1 voice: 0.93
F1 environment: 0.97

####################################
#
# 20180817_orgG527_100_20ep2
# mic4
# testData_G428
#
####################################

                   m       v       e |  sens   acc
music          19733     865      41 |  0.96  0.94
voice            760   13302     374 |  0.92  0.96
environment     1149      93   11983 |  0.91  0.97
--------------------------------------
prec            0.91    0.93    0.97

F1 overall: 0.93
F1 music: 0.93
F1 voice: 0.93
F1 environment: 0.94

####################################
#

                   m       v       e |  sens   acc
music          19975     636      28 |  0.97  0.93
voice           1277   12898     261 |  0.89  0.95
environment     1460      57   11708 |  0.89  0.96
--------------------------------------
prec            0.88    0.95    0.98

F1 overall: 0.92
F1 music: 0.92
F1 voice: 0.92
F1 environment: 0.93

####################################
#
# 20180817_orgG527_450_400_350_300_250_200_150_100_50_21ep4
# mic4
# testData_G428
#
####################################

                   m       v       e |  sens   acc
music          19937     662      40 |  0.97  0.94
voice           1478   12621     337 |  0.87  0.95
environment      736     109   12380 |  0.94  0.97
--------------------------------------
prec            0.90    0.94    0.97

F1 overall: 0.93
F1 music: 0.93
F1 voice: 0.91
F1 environment: 0.95

####################################
#
# 20180817_orgG527_450_400_350_300_250_200_150_100_50_21ep5
# mic4
# testData_G428
#
##############

                   m       v       e |  sens   acc
music           8793     113      11 |  0.99  0.99
voice             75    2779      12 |  0.97  0.99
environment        5       0    2486 |  1.00  1.00
--------------------------------------
prec            0.99    0.96    0.99

F1 overall: 0.98
F1 music: 0.99
F1 voice: 0.97
F1 environment: 0.99

####################################
#
# 20180817_orgG527_450_400_350_300_250_200_150_100_50_21ep1
# mic4
# testData_G527
#
####################################

                   m       v       e |  sens   acc
music           8689     216      12 |  0.97  0.97
voice            145    2717       4 |  0.95  0.97
environment        3       0    2488 |  1.00  1.00
--------------------------------------
prec            0.98    0.93    0.99

F1 overall: 0.97
F1 music: 0.98
F1 voice: 0.94
F1 environment: 1.00

####################################
#
# 20180817_orgG527_450_400_350_300_250_200_150_100_50_21ep2
# mic4
# testData_G527
#
##############

####################################
#
# 20180817_orgG527_400_300_200_100_50_20_10ep2
# mic4
# testData_Studio
#
####################################

                   m       v       e |  sens   acc
music          46546   13991    3058 |  0.73  0.88
voice            261   35964    1679 |  0.95  0.87
environment       16    2143   40178 |  0.95  0.95
--------------------------------------
prec            0.99    0.69    0.89

F1 overall: 0.85
F1 music: 0.84
F1 voice: 0.80
F1 environment: 0.92

####################################
#
# 20180817_orgG527_400_300_200_100_50_20_10ep3
# mic4
# testData_Studio
#
####################################

                   m       v       e |  sens   acc
music          49210   12490    1895 |  0.77  0.90
voice            257   36050    1597 |  0.95  0.89
environment       18    1208   41111 |  0.97  0.97
--------------------------------------
prec            0.99    0.72    0.92

F1 overall: 0.88
F1 music: 0.87
F1 voice: 0.82
F1 environment: 0.95

### trainen van model obv: Studio

In [22]:
baseModelFilename = '20180817_orgStudio'
modelFilePath = storageFolder
logPrefix = 'Orgs Studio'
orgWavDirs = orgsStudio
layersList = [[100, 20], [400, 250, 100, 20], [400, 300, 200, 100, 50, 20, 10], 
              [450, 400, 350, 300, 250, 200, 150, 100, 50, 21]]
nrEpochs = 5
nrsEpochs = [1,2,3,4,5]

In [23]:
# Train voor alle lagen
X_train, Y_train, realTrainClasses = getTrainDataFromFolders(orgWavDirs)
for layers in layersList:
    LOG_HEADER(logPrefix + ', lagen: {}'.format(str(layers)), True)
    train_and_evaluate_per_epoch(X_train, Y_train, realTrainClasses, layers, nrEpochs, modelFilePath, baseModelFilename)

####################################
#
# Orgs Studio, lagen: [100, 20]
#
####################################


*****************
* Epoch nr 1
*****************

Epoch 1/1
353976/353976 [==============================] - 165s 465us/step - loss: 0.2317 - acc: 0.9104
                   m       v       e |  sens   acc
music         114072    8682     136 |  0.93  0.97
voice           3183  115680    3613 |  0.94  0.94
environment      277    5613  102720 |  0.95  0.97
--------------------------------------
prec            0.97    0.89    0.96

F1 overall: 0.94
F1 music: 0.95
F1 voice: 0.92
F1 environment: 0.96


*****************
* Epoch nr 2
*****************

Epoch 1/1
353976/353976 [==============================] - 95s 270us/step - loss: 0.1507 - acc: 0.9430
                   m       v       e |  sens   acc
music         116129    6688      73 |  0.94  0.97
voice           3623  115707    3146 |  0.94  0.95
environment      457    4636  103517 |  0.95  0.98
--------------------------

353976/353976 [==============================] - 243s 688us/step - loss: 0.1991 - acc: 0.9262
                   m       v       e |  sens   acc
music         115885    6883     122 |  0.94  0.97
voice           2642  114566    5268 |  0.94  0.95
environment      272    2814  105524 |  0.97  0.98
--------------------------------------
prec            0.98    0.92    0.95

F1 overall: 0.95
F1 music: 0.96
F1 voice: 0.93
F1 environment: 0.96


*****************
* Epoch nr 2
*****************

Epoch 1/1
353976/353976 [==============================] - 222s 627us/step - loss: 0.1416 - acc: 0.9484
                   m       v       e |  sens   acc
music         115928    6741     221 |  0.94  0.97
voice           2028  116705    3743 |  0.95  0.96
environment      114    3169  105327 |  0.97  0.98
--------------------------------------
prec            0.98    0.92    0.96

F1 overall: 0.96
F1 music: 0.96
F1 voice: 0.94
F1 environment: 0.97


*****************
* Epoch nr 3
*****************



In [24]:
# Test alle modellen

# model parameters
modelFilePath = storageFolder
layerss = layersList

# test data
micNr = 4
testFileNames = ['testData_G428', 'testData_G527', 'testData_Studio']
keyname = 'mic{}'.format(micNr)

for testFileName in testFileNames:
    for layers in layerss:
        for nrEpochs in nrsEpochs:
            testSpecs, testClasses = retrieveTestData(testFileName, keyname)

            LOG_HEADER(getModelName(baseModelFilename, layers, nrEpochs) 
                       + '\n# ' + keyname
                       + '\n# ' + testFileName,
                       True)
            matrix = evaluate_model(testSpecs, testClasses, layers, nrEpochs, modelFilePath, baseModelFilename)


####################################
#
# 20180817_orgStudio_100_20ep1
# mic4
# testData_G428
#
####################################

                   m       v       e |  sens   acc
music          20521     117       1 |  0.99  0.77
voice           5987    8326     123 |  0.58  0.87
environment     4917      32    8276 |  0.63  0.89
--------------------------------------
prec            0.65    0.98    0.99

F1 overall: 0.76
F1 music: 0.79
F1 voice: 0.73
F1 environment: 0.77

####################################
#
# 20180817_orgStudio_100_20ep2
# mic4
# testData_G428
#
####################################

                   m       v       e |  sens   acc
music          20492     147       0 |  0.99  0.72
voice           6352    7994      90 |  0.55  0.86
environment     6809      17    6399 |  0.48  0.86
--------------------------------------
prec            0.61    0.98    0.99

F1 overall: 0.70
F1 music: 0.75
F1 voice: 0.71
F1 environment: 0.65

##################################

                   m       v       e |  sens   acc
music          20499     132       8 |  0.99  0.82
voice           5436    8852     148 |  0.61  0.88
environment     3141       9   10075 |  0.76  0.93
--------------------------------------
prec            0.71    0.98    0.98

F1 overall: 0.81
F1 music: 0.82
F1 voice: 0.76
F1 environment: 0.86

####################################
#
# 20180817_orgStudio_450_400_350_300_250_200_150_100_50_21ep4
# mic4
# testData_G428
#
####################################

                   m       v       e |  sens   acc
music          20381     253       5 |  0.99  0.80
voice           4017   10307     112 |  0.71  0.91
environment     5513      38    7674 |  0.58  0.88
--------------------------------------
prec            0.68    0.97    0.98

F1 overall: 0.79
F1 music: 0.81
F1 voice: 0.82
F1 environment: 0.73

####################################
#
# 20180817_orgStudio_450_400_350_300_250_200_150_100_50_21ep5
# mic4
# testData_G428
#
##########

                   m       v       e |  sens   acc
music           8914       3       0 |  1.00  0.91
voice            887    1978       1 |  0.69  0.94
environment      386       0    2105 |  0.85  0.97
--------------------------------------
prec            0.88    1.00    1.00

F1 overall: 0.89
F1 music: 0.93
F1 voice: 0.82
F1 environment: 0.92

####################################
#
# 20180817_orgStudio_450_400_350_300_250_200_150_100_50_21ep1
# mic4
# testData_G527
#
####################################

                   m       v       e |  sens   acc
music           8912       5       0 |  1.00  0.85
voice           1517    1337      12 |  0.47  0.89
environment      675       0    1816 |  0.73  0.95
--------------------------------------
prec            0.80    1.00    0.99

F1 overall: 0.79
F1 music: 0.89
F1 voice: 0.64
F1 environment: 0.84

####################################
#
# 20180817_orgStudio_450_400_350_300_250_200_150_100_50_21ep2
# mic4
# testData_G527
#
##########

####################################
#
# 20180817_orgStudio_400_300_200_100_50_20_10ep2
# mic4
# testData_Studio
#
####################################

                   m       v       e |  sens   acc
music          59820    3584     191 |  0.94  0.96
voice           1533   35832     539 |  0.95  0.96
environment      203     343   41791 |  0.99  0.99
--------------------------------------
prec            0.97    0.90    0.98

F1 overall: 0.95
F1 music: 0.96
F1 voice: 0.92
F1 environment: 0.98

####################################
#
# 20180817_orgStudio_400_300_200_100_50_20_10ep3
# mic4
# testData_Studio
#
####################################

                   m       v       e |  sens   acc
music          60054    3444      97 |  0.94  0.96
voice           1967   35643     294 |  0.94  0.96
environment      123     453   41761 |  0.99  0.99
--------------------------------------
prec            0.97    0.90    0.99

F1 overall: 0.95
F1 music: 0.96
F1 voice: 0.92
F1 environment: 